In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
import sys
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

import requests
from selenium import webdriver
from selenium.webdriver.common.by import By

from tensorflow.keras.preprocessing.text import Tokenizer
from scipy.spatial.distance import cosine

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: use options instead of chrome_options


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Подбор навыков под заданную специализацию

Предлагаем пользователю ввести ФИО и выбрать специализацию.  
По специализации парсим вакансии.

In [ ]:
data = pd.read_json('/content/drive/MyDrive/Хакатон/MCH/data.json')
data.head()

,id,name,salary,published_at,experience,key_skills
0,44240210,Data Scientist,150000.0,2021-05-24 18:04:26,notSpecified,"[АНАЛИЗ, PYTHON, C]"
1,44872776,Junior Data Scientist,50000.0,2021-05-25 11:10:42,notSpecified,"[PANDAS, PYTHON, SQL]"
2,44350010,Data Scientist,100000.0,2021-05-25 06:43:26,notSpecified,"[XGBOOST, CATBOOST, SCIKIT-LEARN, C]"
3,44322933,Data Science Manager,288240.0,2021-05-24 11:36:45,notSpecified,"[MLOPS, DESIGN, COLLECTION]"
4,44322935,Data Science Manager,288240.0,2021-05-24 11:36:45,notSpecified,"[MLOPS, DESIGN, COLLECTION]"


Получаем самые востребованные навыки в данной сфере.

In [ ]:
def get_top_skills(vacancy_data):
    skills = vacancy_data['key_skills'].copy()

    tokenizer = Tokenizer(num_words=400)
    tokenizer.fit_on_texts(skills)

    top_skills = list(tokenizer.index_word.values())[:10]

    return top_skills

In [ ]:
top_skills = get_top_skills(data)
top_skills

['python',
 'sql',
 'data mining',
 'английский язык',
 'c',
 'c++',
 'linux',
 'data science',
 'java',
 'hadoop']

Получаем от пользователя информацию о его уровне по этим навыкам (в том числе предлагаем пройти тестирование)

In [ ]:
def check_skills(skills):
    skills_level = {}
    for skill in skills:
        level = int(input(f'Ваш уровень по навыку {skill}: '))
        skills_level[skill] = level

    return skills_level

In [ ]:
user_skills_level = check_skills(top_skills)
user_skills_level

Ваш уровень по навыку python: 2
Ваш уровень по навыку sql: 1
Ваш уровень по навыку data mining: 1
Ваш уровень по навыку английский язык: 1
Ваш уровень по навыку c: 0
Ваш уровень по навыку c++: 0
Ваш уровень по навыку linux: 0
Ваш уровень по навыку data science: 1
Ваш уровень по навыку java: 0
Ваш уровень по навыку hadoop: 0


{'c': 0,
 'c++': 0,
 'data mining': 1,
 'data science': 1,
 'hadoop': 0,
 'java': 0,
 'linux': 0,
 'python': 2,
 'sql': 1,
 'английский язык': 1}

Находим подходящие курсы по навыку и текущему уровню пользователя

## Парсинг Нетологии

In [ ]:
def find_netology_courses(query, level=1):
    level = {0: 'neo', 1: 'neo', 2: 'pro', 3: 'pro'}[level]

    url = "https://netology.ru/backend/api/search/programs"
    response = requests.get(url, params=[('term', query)]).json()['programs']
    courses = list(filter(lambda x: x['educational_level'] is not None and x['educational_level']['slug'] == level, response))
    return pd.DataFrame(courses)

In [ ]:
all_courses = {}
for skill, level in user_skills_level.items():
    courses = find_netology_courses(skill, level=level)
    all_courses[skill] = courses

In [ ]:
all_courses['sql']

,id,type,program_type,label,name,url,duration,projects_amount,experts_amount,starts_at,date,program_image,color,slider_description,url_to_replace,landing_url,tags,navigational_tag_ids,program_family_id,program_type_id,program_type_alias,is_async,initial_price,current_price,program_family_image,educational_level,starting_soon,main_direction,show_direction,directions,partner_logo,price_type,description
0,19960,online,course,Осталось 4 дня,SQL и получение данных,//netology.ru/programs/sql-lessons,6 недель,21 занятие,5 экспертов,17 июня,2021-06-17,//netology.ru/program/sql-lessons.png,#7EC9E4,,,,[],[],643,2,course,False,28900,20230,{'url': 'https://u.netology.ngcdn.ru/backend/u...,"{'id': 1, 'name': 'Нео', 'slug': 'neo'}",True,"{'slug': 'data-science', 'name': 'Аналитика'}","{'slug': 'data-science', 'name': 'Аналитика'}","[{'slug': 'data-science', 'name': 'Аналитика'}...",,paid,<p>SQL &mdash; главный инструмент аналитика. Н...
1,21391,online,lesson,Осталось 25 дней,"SQL для аналитиков, маркетологов и product-мен...",//netology.ru/free-lessons/zachem-analitikam-sql,0 дней,1 занятие,1 эксперт,8 июля,2021-07-08,,#50b9e1,,,None,[],[],834,1,lesson,False,0,0,{'url': None},"{'id': 1, 'name': 'Нео', 'slug': 'neo'}",False,"{'slug': 'data-science', 'name': 'Аналитика'}",NaN,"[{'slug': 'data-science', 'name': 'Аналитика'}]",,free,"Расскажем, чем SQL полезен аналитику, маркетол..."


## Парсинг Курсеры

In [ ]:
# здесь продублировал уровень Advanced, поскольку у нас 4 уровня

def find_coursera_courses(driver, query, page=1, level=1):
    """
    level - 1, 2, 3 (Beginner, Intermediate, Advanced)
    """

    level = {0: 'Beginner', 1: 'Intermediate', 2: 'Advanced', 3: 'Advanced'}[level]

    if page == 1:
        driver.get(f'https://ru.coursera.org/search?query={query}&entityTypeDescription=Courses&index=prod_all_products_term_optimization&productDifficultyLevel={level}&allLanguages=Russian')
    else:
        driver.get(f'https://ru.coursera.org/search?query={query}&entityTypeDescription=Courses&page={page}&index=prod_all_products_term_optimization&productDifficultyLevel={level}&allLanguages=Russian')
    time.sleep(2)
    driver.save_screenshot('img.png')
    elements = driver.find_elements(By.CLASS_NAME, 'ais-InfiniteHits-item')
    print(f'Found {len(elements)} courses')
    courses = []
    for i, element in enumerate(elements):
        element.screenshot(f'img{i}.png')
        # print(i)
        try:
            link = element.find_element(By.TAG_NAME, 'a').get_attribute('href')
            title = element.find_element(By.CLASS_NAME, 'card-title').text
            difficulty = element.find_element(By.CLASS_NAME, 'difficulty').text
            author = element.find_element(By.CLASS_NAME, 'partner-name').text
            rate = element.find_element(By.CLASS_NAME, 'ratings-text').text

            courses.append({
                'title': title,
                'difficulty': difficulty,
                'rate': rate,
                'author': author,
                'link': link,
            })
        except Exception as e:
            # print(e)
            pass
    data = pd.DataFrame(courses)
    return data

In [ ]:
all_courses = {}
for skill, level in user_skills_level.items():
    courses = find_coursera_courses(driver, skill, page=1, level=level)
    all_courses[skill] = courses

Found 10 courses
Found 10 courses
Found 10 courses
Found 8 courses
Found 10 courses
Found 10 courses
Found 10 courses
Found 10 courses
Found 10 courses
Found 10 courses


In [ ]:
all_courses['python']

,title,difficulty,rate,author,link
0,Python for Everybody,Beginner,4.8,University of Michigan,https://ru.coursera.org/specializations/python
1,Google IT Automation with Python,Beginner,4.7,Google,https://ru.coursera.org/professional-certifica...
2,Python 3 Programming,Beginner,4.7,University of Michigan,https://ru.coursera.org/specializations/python...
3,Applied Data Science with Python,Intermediate,4.5,University of Michigan,https://ru.coursera.org/specializations/data-s...
4,Crash Course on Python,Beginner,4.8,Google,https://ru.coursera.org/learn/python-crash-course
5,IBM Data Science,Beginner,4.6,IBM,https://ru.coursera.org/professional-certifica...
6,Programming for Everybody (Getting Started wit...,Mixed,4.8,University of Michigan,https://ru.coursera.org/learn/python
7,Data Science Fundamentals with Python and SQL,Beginner,4.6,IBM,https://ru.coursera.org/specializations/data-s...
8,IBM Data Analyst,Beginner,4.6,IBM,https://ru.coursera.org/professional-certifica...


# Подбор вакансии по навыкам пользователя

Также предлагаем пользователю ввести ФИО и выбрать специализацию. И сразу получаем от него навыки и их оценки.

In [ ]:
user_data = {'name': 'Иванов Иван Иванович',
             'specialization': 'Data Science',
             'skills': {'PYTHON': 2,
                        'CATBOOST': 1,
                        'PANDAS': 2,
                        'TENSORFLOW': 1}
             }

Парсим вакансии и считаем косинусное расстояние между навыками пользователя и каждой вакансией.  
Возвращаем отранжированный список.

In [ ]:
def calculate_distance(skills, user_skills):
    cos_dist = []
    for s in skills:
        cos_dist.append(cosine(user_skills, s))
    cos_dist = np.array(cos_dist)
    return cos_dist

In [ ]:
def get_actual_vacancy(vacancy_data, user_data):
    skills = vacancy_data['key_skills'].copy()

    # токенизация навыков
    tokenizer = Tokenizer(num_words=400)
    tokenizer.fit_on_texts(skills)

    bow_skills = tokenizer.texts_to_matrix(skills)
    user_skills = list(user_data['skills'].keys())
    bow_user_skills = tokenizer.texts_to_matrix([user_skills])

    # расчет косинусного расстояния навыков пользователя до навыков вакансий
    cos_dist = calculate_distance(bow_skills, bow_user_skills)

    # выбор актуальных вакансий
    actual_vacancy = vacancy_data.iloc[np.argsort(cos_dist)[:5]]

    return actual_vacancy

In [ ]:
actual_vacancy = get_actual_vacancy(data, user_data)

Показываем пользователю вакансии

In [ ]:
actual_vacancy

,id,name,salary,published_at,experience,key_skills
1,44872776,Junior Data Scientist,50000.0,2021-05-25 11:10:42,notSpecified,"[PANDAS, PYTHON, SQL]"
23,44413603,Ведущий аналитик (Data Science),NaN,2021-05-26 12:50:19,notSpecified,"[ERP, PANDAS, PYTHON]"
266,38795381,Преподаватель курса по направлению Data Science,NaN,2021-05-12 08:38:50,noExperience,[PYTHON]
379,43505794,Python developer with Big Data (office/remote),NaN,2021-04-28 14:28:56,between3And6,[PYTHON]
387,44364228,Senior Data Engineer OR Machine Learning,NaN,2021-05-04 12:18:38,noExperience,"[PYTHON, TENSORFLOW, KERAS, АНГЛИЙСКИЙ ЯЗЫК]"


По каждой вакансии есть возможность посмотреть какие курсы можно пройти, чтобы еще лучше ей соответствовать.  
Для этого также парсим образовательные платформы по навыкам пользователя + недостающим навыкам из вакансии.  

Можно добавить возможность дооценить недостающие навыки, если у пользователя они есть, но ранее он их не указал.

In [ ]:
update_skills = user_data['skills']
for skill in actual_vacancy.key_skills[1]:
    if skill not in update_skills:
        update_skills[skill] = 0

In [ ]:
update_skills

{'CATBOOST': 1, 'PANDAS': 2, 'PYTHON': 2, 'SQL': 0, 'TENSORFLOW': 1}

Нетология

In [ ]:
all_courses = {}
for skill, level in update_skills.items():
    courses = find_netology_courses(skill, level=level)
    all_courses[skill] = courses

In [ ]:
all_courses['PYTHON']

,id,type,program_type,label,name,url,duration,projects_amount,experts_amount,starts_at,date,program_image,color,slider_description,url_to_replace,landing_url,tags,navigational_tag_ids,program_family_id,program_type_id,program_type_alias,is_async,initial_price,current_price,program_family_image,educational_level,starting_soon,main_direction,directions,partner_logo,price_type,description
0,21376,online,specialization,Осталось 25 дней,DevOps-инженер,//netology.ru/programs/devops,11 месяцев,23 занятия,0 экспертов,8 июля,2021-07-08,,#F6BD94,,,,[],[],1209,4,specialization,False,87900,70740,{'url': 'https://u.netology.ngcdn.ru/backend/u...,"{'id': 2, 'name': 'Про', 'slug': 'pro'}",False,"{'slug': 'development', 'name': 'Программирова...","[{'slug': 'development', 'name': 'Программиров...",,paid,<p>- Освойте одну из самых востребованных IT-п...


Coursera

In [ ]:
all_courses = {}
for skill, level in update_skills.items():
    courses = find_coursera_courses(driver, skill, page=1, level=level)
    all_courses[skill] = courses

Found 10 courses
Found 21 courses
Found 10 courses
Found 10 courses
Found 10 courses


In [ ]:
all_courses['PANDAS']

,title,difficulty,rate,author,link
0,Applied Data Science with Python,Intermediate,4.5,University of Michigan,https://ru.coursera.org/specializations/data-s...
1,IBM Data Analyst,Beginner,4.6,IBM,https://ru.coursera.org/professional-certifica...
2,Python for Data Analysis: Pandas & NumPy,Beginner,4.2,Coursera Project Network,https://ru.coursera.org/projects/python-for-da...
3,Exploratory Data Analysis With Python and Pandas,Beginner,4.6,Coursera Project Network,https://ru.coursera.org/projects/exploratory-d...
4,Introduction to Data Science in Python,Intermediate,4.5,University of Michigan,https://ru.coursera.org/learn/python-data-anal...
5,COVID19 Data Analysis Using Python,Intermediate,4.6,Coursera Project Network,https://ru.coursera.org/projects/covid19-data-...
6,Applied Data Science,Beginner,4.6,IBM,https://ru.coursera.org/specializations/applie...
7,Data Analysis with Python,Beginner,4.7,IBM,https://ru.coursera.org/learn/data-analysis-wi...
8,Data Analysis Using Python,Beginner,4.5,University of Pennsylvania,https://ru.coursera.org/learn/data-analysis-py...
9,Python Pandas Basics: Load and Export Data,Beginner,4.4,Coursera Project Network,https://ru.coursera.org/projects/python-pandas...
